In [2]:
import pandas as pd
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset, load_from_disk
import numpy as np
import nltk
nltk.download('punkt')
import pandas as pd
from datasets import Dataset, DatasetDict
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from transformers import pipeline



2024-03-13 19:40:40.896310: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-13 19:40:41.242774: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-13 19:40:58.570708: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[nltk_data] Downloading package punkt to /home/nair.ro/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
train_df = pd.read_csv('/work/LitArt/data/generated_summaries/train_dataset_with_summaries.csv')
test_df = pd.read_csv('/work/LitArt/data/generated_summaries/test_dataset_with_summaries.csv')
val_df = pd.read_csv('/work/LitArt/data/generated_summaries/validation_dataset_with_summaries.csv')


In [8]:
train_df.head()

,chapter,summary_text,__index_level_0__,generated_summary
0,one evening as i was lying flat on the deck of...,from the deck of his steamboat marlow overhear...,5664,"On a steamboat, the narrator overhears a conve..."
1,the river--seemed to beckon with a dishonourin...,from the deck of his steamboat marlow overhear...,5664,"Narrator travels up a mysterious river, encoun..."
2,air for a while--and on we went again into the...,from the deck of his steamboat marlow overhear...,5664,The passage describes a journey through a dens...
3,instructed and what he knew was this--that sho...,from the deck of his steamboat marlow overhear...,5664,"A fireman, fearing an evil spirit in the boile..."
4,we were till next morning moreover he pointed ...,from the deck of his steamboat marlow overhear...,5664,"Delayed by sensible caution, the narrator navi..."


In [11]:
def pre_processing(sentence):
    #lower text
    sentence = sentence.lower()

    pattern = r"\s*\([a-zA-Z]\s_\)"
    sentence = re.sub(pattern, "", sentence)
        
    sentence = sentence.replace("\n"," ")
    
    # replacing everything with space  
    sentence = re.sub(r"[=.!,¿?.!+,;¿/:|%()<>।॰{}#_'\"@$^&*']", " ", sentence)
    sentence = re.sub(r"…", " ", sentence)
    
    #remove double quotes
    sentence = re.sub(r'"', " ", sentence)
    
    #remove numbers
    sentence = re.sub(r'[0-9]', "", sentence)
    #sentence = re.sub(r'#([^s]+)', r'1', sentence)
    
    #remove website links
    sentence = re.sub('((www.[^s]+)|(https?://[^s]+))','',sentence)
    
    #remove @anythin here
    #sentence = re.sub('@[^s]+','',sentence)
    
    #remove multiple spaces
    sentence = re.sub(r'[" "]+', " ", sentence)
    
    # remove extra space
    sentence = sentence.strip()
    
    return sentence

In [13]:

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
val_dataset = Dataset.from_pandas(val_df)

Dataset_dic = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset,
    "test": test_dataset
})




In [29]:
def show_samples(dataset, num_samples=5, seed=42):
    sample = dataset["train"].shuffle(seed=seed).select(range(num_samples))
    for example in sample:
        print(f"\n'>> chapter: {example['chapter'][:500]}'")
        print(f"'>> generated_summary: {example['generated_summary'][:500]}'")
        print("\n")


show_samples(Dataset_dic)


'>> chapter: venture and with a little portion of emmy s savings besides her pension of fifty pounds a year there had been five hundred pounds as her husband s executor stated left in the agent s hands at the time of osborne s demise which sum as george s guardian dobbin proposed to put out at per cent in an indian house of agency mr sedley who thought the major had some roguish intentions of his own about the money was strongly against this plan and he went to the agents to protest personally against the em'
'>> generated_summary: Emmy's savings and pension were left by her late husband in Major Dobbin's care, leading to misunderstandings and reconciliations.'



'>> chapter: xv knitting there had been earlier drinking than usual in the wine-shop of monsieur defarge as early as six o clock in the morning sallow faces peeping through its barred windows had descried other faces within bending over measures of wine monsieur defarge sold a very thin wine at the best of times but it would

In [17]:
df_new = pd.concat([train_df, test_df , val_df], axis=0)

In [22]:
len(df_new)

13830

In [23]:

df_new['chapter_word_count'] = df_new['chapter'].apply(lambda x: len(x.split()))
df_new['summary_text_word_count'] = df_new['generated_summary'].apply(lambda x: len(x.split()))

average_chapter_words = df_new['chapter_word_count'].mean()
max_chapter_words = df_new['chapter_word_count'].max()
median_chapter_words = df_new['chapter_word_count'].median()

average_summary_text_words = df_new['summary_text_word_count'].mean()
max_summary_text_words = df_new['summary_text_word_count'].max()
median_summary_text_words = df_new['summary_text_word_count'].median()



In [24]:
    print("average_summary_text_words = ", average_summary_text_words ) 
    print(" max_summary_text_words = " , max_summary_text_words)
    print("median_summary_text_words = " , median_summary_text_words)
    print("average_chapter_words = " , average_chapter_words)
    print("max_chapter_words = " ,max_chapter_words )
    print(" median_chapter_words = " ,median_chapter_words )

average_summary_text_words =  23.425090383224873
 max_summary_text_words =  143
median_summary_text_words =  20.0
average_chapter_words =  888.3359363702097
max_chapter_words =  1000
 median_chapter_words =  1000.0


In [25]:

bottom_10_threshold_chapter = df_new['chapter_word_count'].quantile(0.1)
top_10_threshold_chapter = df_new['chapter_word_count'].quantile(0.9)

bottom_10_threshold_summary_text = df_new['summary_text_word_count'].quantile(0.1)
top_10_threshold_summary_text = df_new['summary_text_word_count'].quantile(0.9)

outlier_rows = df_new[(df_new['chapter_word_count'] < bottom_10_threshold_chapter) | 
                      (df_new['chapter_word_count'] > top_10_threshold_chapter) |
                      (df_new['summary_text_word_count'] < bottom_10_threshold_summary_text) | 
                      (df_new['summary_text_word_count'] > top_10_threshold_summary_text)]

number_of_outliers = outlier_rows.shape[0]

df_without_outliers = df_new[~((df_new['chapter_word_count'] < bottom_10_threshold_chapter) | 
                               (df_new['chapter_word_count'] > top_10_threshold_chapter) |
                               (df_new['summary_text_word_count'] < bottom_10_threshold_summary_text) | 
                               (df_new['summary_text_word_count'] > top_10_threshold_summary_text))]

print("Number of outlier rows: " , number_of_outliers)
print(" bottom_10_threshold_chapter = " ,df_new[(df_new['chapter_word_count'] < bottom_10_threshold_chapter)].shape[0] )
print(" top_10_threshold_chapter = " ,df_new[(df_new['chapter_word_count'] > top_10_threshold_chapter)].shape[0] )
print("top_10_threshold_summary_text = " ,df_new[(df_new['summary_text_word_count'] > top_10_threshold_summary_text)].shape[0] )
print("bottom_10_threshold_summary_text = " ,df_new[(df_new['summary_text_word_count'] < bottom_10_threshold_summary_text)].shape[0] )



Number of outlier rows:  3404
 bottom_10_threshold_chapter =  1383
 top_10_threshold_chapter =  0
top_10_threshold_summary_text =  1344
bottom_10_threshold_summary_text =  903


In [26]:

df_without_outliers['chapter_word_count'] = df_without_outliers['chapter'].apply(lambda x: len(x.split()))
df_without_outliers['summary_text_word_count'] = df_without_outliers['generated_summary'].apply(lambda x: len(x.split()))

average_chapter_words = df_without_outliers['chapter_word_count'].mean()
max_chapter_words = df_without_outliers['chapter_word_count'].max()
median_chapter_words = df_without_outliers['chapter_word_count'].median()

average_summary_text_words = df_without_outliers['summary_text_word_count'].mean()
max_summary_text_words = df_without_outliers['summary_text_word_count'].max()
median_summary_text_words = df_without_outliers['summary_text_word_count'].median()

print("After removing outliers \n")
print("average_summary_text_words = ", average_summary_text_words ) 
print(" max_summary_text_words = " , max_summary_text_words)
print("median_summary_text_words = " , median_summary_text_words)
print("average_chapter_words = " , average_chapter_words)
print("max_chapter_words = " ,max_chapter_words )
print(" median_chapter_words = " ,median_chapter_words )



After removing outliers 

average_summary_text_words =  21.338001150968733
 max_summary_text_words =  31
median_summary_text_words =  20.0
average_chapter_words =  961.5292537886054
max_chapter_words =  1000
 median_chapter_words =  1000.0


/tmp/ipykernel_237385/2713342462.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_without_outliers['chapter_word_count'] = df_without_outliers['chapter'].apply(lambda x: len(x.split()))
/tmp/ipykernel_237385/2713342462.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_without_outliers['summary_text_word_count'] = df_without_outliers['generated_summary'].apply(lambda x: len(x.split()))
